In [ ]:
import os
from os.path import join as ojoin

In [ ]:
sim_dir = ojoin('..', 'simulation_results') 

In [ ]:
problems = dict()
problems['fairim'] = ['group_maximin', 'dc']
problems['mip'] = problems['fairim'] + ['equality', 'equity']

In [ ]:
def extract_coverage(fpath):
    total_coverage = 0
    with open(fpath) as f:
        for line in f:
            line = line.strip()
            if not line: continue
            total_coverage += float(line)
    return total_coverage

In [ ]:
coverage_dict = dict()

tim_coverage_list = []
for graph_number in range(20):
    coverage = extract_coverage(ojoin(sim_dir, 'tim', 'output_%d.txt'%graph_number))
    if coverage > 0:
        tim_coverage_list.append(coverage)
coverage_dict['tim'] = tim_coverage_list

In [ ]:
for approach in ('mip', 'fairim'):
    for problem in problems[approach]:
        for attribute in ('age', 'gender', 'ethnicity'):
            output_dir = ojoin(sim_dir, approach, problem, attribute)
            coverage_list = []
            for graph_number in range(20):
                coverage = extract_coverage(ojoin(output_dir, 'output_%d.txt'%graph_number))
                if coverage > 0:
                    coverage_list.append(coverage)
            coverage_dict[approach, problem, attribute] = coverage_list

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np



def draw(values, labels, colors, title, ylabel, fpath, ylim=None, figsize=None):
    if figsize is None:
        figsize=(3.5, 2)
    plt.figure(figsize=figsize)
    means = [np.mean(v) for v in values]
    stds = [np.std(v) for v in values]
    xpos = range(len(labels))

    bars = plt.bar(xpos, means, yerr=stds, width=0.5, align='center', capsize=4)
    for i in xpos:
        bars[i].set_color(colors[i])
    _ = plt.xticks(xpos, labels, rotation=0, fontsize=10)
    if ylim:
        _ = plt.ylim(ylim)
    _ = plt.ylabel(ylabel, fontsize=10)
    _ = plt.title(title, fontsize=12)
    _ = plt.savefig(fpath, bbox_inches='tight')
    

### Plots 1 to 3
In the first three plot we present the total coverage obtained by these methods, for groups *age*, *gender*, and *ethinicity*:
- tim
- equity (MIP)
- equality (MIP)
- group maximin (MIP)
- DC (MIP)



In [ ]:
labels = ['tim', 'equality', 'equity', 'maximin', 'diversity']
colors = ['#606060'] + ['#bababa'] * 4

for attribute in ('age', 'gender', 'ethnicity'):
    keys = ['tim', ('mip', 'equality', attribute), ('mip', 'equity', attribute), 
            ('mip', 'group_maximin', attribute), ('mip', 'dc', attribute)]
    values = [np.array(coverage_dict[key]) for key in keys]
    title = attribute[0].upper() + attribute[1:]
    fpath = 'images/mip_coverage/%s.pdf'%attribute
    draw(values, labels, colors, title, 'Coverage', fpath, ylim=(60, 85), figsize=(4, 2.5))

### Plot 2 to 6
In the second plot we present the total coverage obtained by these methods, for groups *age*, *gender*, and *ethinicity*:
- group maximin (MIP)
- group maximin (fairIM)
- DC (MIP)
- DC (fairIM)

In [ ]:
labels = ['maximin\n(fairIM)', 'maximin\n(MIP)', 'diversity\n(fairIM)', 'diversity\n(MIP)']
colors = ['#606060', '#bababa'] * 2

for attribute in ('age', 'gender', 'ethnicity'):
    keys = [('fairim', 'group_maximin', attribute), 
            ('mip', 'group_maximin', attribute), 
            ('fairim', 'dc', attribute), 
            ('mip', 'dc', attribute)]
    values = [np.array(coverage_dict[key]) for key in keys]
    title = attribute[0].upper() + attribute[1:]
    fpath = 'images/mip_vs_fairim/%s.pdf'%attribute
    draw(values, labels, colors, title, 'Coverage', fpath, ylim=(60, 85))